In [1]:
import openeo

In [2]:
connection = openeo.connect("http://localhost:8000").authenticate_basic("user", "password")

In [ ]:
datacube_crop = connection.load_collection("sentinel-s2-l2a-cogs",
    spatial_extent={"west": -4.026661,"south": 48.202738,"east": -3.743587,"north": 48.300371,"crs": 4326},
    temporal_extent=["2017-03-01", "2017-05-30"],
    bands=["B02", "B03", "B04", "B08"]
)

In [ ]:

datacube_aoi = connection.load_collection("sentinel-s2-l2a-cogs",
    spatial_extent={"west": -2.997065,"south": 47.892255,"east": -2.671039,"north": 48.047242,"crs": 4326},
    temporal_extent=["2017-03-01", "2017-05-30"],
    bands=["B02", "B03", "B04", "B08"]
)

In [5]:
datacube_crop = datacube_crop.aggregate_temporal_period(period="month", reducer="median")
datacube_aoi = datacube_aoi.aggregate_temporal_period(period="month", reducer="median")

In [6]:
datacube_crop = datacube_crop.ndvi(red="B04", nir="B08", target_band="NDVI")
datacube_aoi = datacube_aoi.ndvi(red="B04", nir="B08", target_band="NDVI")

In [7]:
datacube_crop = datacube_crop.process("array_interpolate_linear", data=datacube_crop)
datacube_aoi = datacube_aoi.process("array_interpolate_linear", data=datacube_aoi)

In [8]:
training_cube = datacube_crop.aggregate_spatial(
    geometries="../train_data/train_data.geojson",reducer="mean"
)

In [9]:
model_rf = connection.mlm_class_random_forest(
    max_variables="sqrt",
    num_trees=200,
    seed=42
)

In [10]:
model = model_rf.fit(training_set=training_cube, target="class_name")

In [11]:
prediction = model.predict(data=datacube_aoi)

In [12]:
final_result = prediction.save_result(format="GTiff")

In [13]:
connection.download(graph=final_result, outputfile="france_prediction.tif", timeout=7200)